In [1]:
from main import DataImporter
from models.modelhelper import ModelHelper
import numpy as np

data_importer = DataImporter()
data_importer.import_data()

ds_train = data_importer.get_train_data()
ds_test = data_importer.get_test_data()
ds_validation = data_importer.get_validation_data()

In [2]:
ds_train.info()
print('\n')
ds_test.info()
print('\n')
ds_validation.info()
print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10269 entries, 0 to 10268
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10269 non-null  object 
 1   label                 10269 non-null  int64  
 2   statement             10269 non-null  object 
 3   subject               10269 non-null  object 
 4   speaker               10269 non-null  object 
 5   job_title             10269 non-null  object 
 6   state_info            10269 non-null  object 
 7   party_affiliation     10269 non-null  object 
 8   barely_true_counts    10269 non-null  float32
 9   false_counts          10269 non-null  float32
 10  half_true_counts      10269 non-null  float32
 11  mostly_true_counts    10269 non-null  float32
 12  pants_on_fire_counts  10269 non-null  float32
 13  context               10269 non-null  object 
dtypes: float32(5), int64(1), object(8)
memory usage: 922.7+ KB


<class 'p

In [3]:
# Drop specified columns from each dataset
columns_to_drop = ['id', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'context']

ds_train = ds_train.drop(columns=columns_to_drop)
ds_test = ds_test.drop(columns=columns_to_drop) 
ds_validation = ds_validation.drop(columns=columns_to_drop)

# Display results
print("Training Dataset:")
display(ds_train.head())
print("\nTest Dataset:") 
display(ds_test.head())
print("\nValidation Dataset:")
display(ds_validation.head())


Training Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,0,Says the Annies List political group supports ...,0.0,1.0,0.0,0.0,0.0
1,1,When did the decline of coal start? It started...,0.0,0.0,1.0,1.0,0.0
2,2,"Hillary Clinton agrees with John McCain ""by vo...",70.0,71.0,160.0,163.0,9.0
3,0,Health care reform legislation is likely to ma...,7.0,19.0,3.0,5.0,44.0
4,1,The economic turnaround started at the end of ...,15.0,9.0,20.0,19.0,2.0



Test Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,3,Building a wall on the U.S.-Mexico border will...,30.0,30.0,42.0,23.0,18.0
1,0,Wisconsin is on pace to double the number of l...,2.0,1.0,0.0,0.0,0.0
2,0,Says John McCain has done nothing to help the ...,63.0,114.0,51.0,37.0,61.0
3,1,Suzanne Bonamici supports a plan that will cut...,1.0,1.0,3.0,1.0,1.0
4,5,When asked by a reporter whether hes at the ce...,5.0,7.0,2.0,2.0,7.0



Validation Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts
0,4,We have less Americans working now than in the...,1.0,0.0,1.0,0.0,0.0
1,5,"When Obama was sworn into office, he DID NOT u...",11.0,43.0,8.0,5.0,105.0
2,0,Says Having organizations parading as being so...,0.0,1.0,1.0,1.0,0.0
3,1,Says nearly half of Oregons children are poor.,0.0,1.0,1.0,1.0,0.0
4,1,On attacks by Republicans that various program...,70.0,71.0,160.0,163.0,9.0


In [4]:
# Combining the counts of 'false_counts' and 'pants_on_fire_counts' into 'false_counts'
ds_train['false_counts'] = ds_train['false_counts'] + ds_train['pants_on_fire_counts']
ds_test['false_counts'] = ds_test['false_counts'] + ds_test['pants_on_fire_counts']
ds_validation['false_counts'] = ds_validation['false_counts'] + ds_validation['pants_on_fire_counts']

In [5]:
# Dropping 'pants_on_fire_counts' from each dataset
ds_train = ds_train.drop('pants_on_fire_counts', axis=1)
ds_test = ds_test.drop('pants_on_fire_counts', axis=1) 
ds_validation = ds_validation.drop('pants_on_fire_counts', axis=1)

# Display results
print("Training Dataset:")
display(ds_train.head())
print("\nTest Dataset:") 
display(ds_test.head())
print("\nValidation Dataset:")
display(ds_validation.head())

Training Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts
0,0,Says the Annies List political group supports ...,0.0,1.0,0.0,0.0
1,1,When did the decline of coal start? It started...,0.0,0.0,1.0,1.0
2,2,"Hillary Clinton agrees with John McCain ""by vo...",70.0,80.0,160.0,163.0
3,0,Health care reform legislation is likely to ma...,7.0,63.0,3.0,5.0
4,1,The economic turnaround started at the end of ...,15.0,11.0,20.0,19.0



Test Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts
0,3,Building a wall on the U.S.-Mexico border will...,30.0,48.0,42.0,23.0
1,0,Wisconsin is on pace to double the number of l...,2.0,1.0,0.0,0.0
2,0,Says John McCain has done nothing to help the ...,63.0,175.0,51.0,37.0
3,1,Suzanne Bonamici supports a plan that will cut...,1.0,2.0,3.0,1.0
4,5,When asked by a reporter whether hes at the ce...,5.0,14.0,2.0,2.0



Validation Dataset:


,label,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts
0,4,We have less Americans working now than in the...,1.0,0.0,1.0,0.0
1,5,"When Obama was sworn into office, he DID NOT u...",11.0,148.0,8.0,5.0
2,0,Says Having organizations parading as being so...,0.0,1.0,1.0,1.0
3,1,Says nearly half of Oregons children are poor.,0.0,1.0,1.0,1.0
4,1,On attacks by Republicans that various program...,70.0,80.0,160.0,163.0


In [6]:
# Combining the counts of 'false_counts' and 'barely_true_counts' into 'false_counts'
ds_train['half_true_counts'] = ds_train['half_true_counts'] + ds_train['barely_true_counts']
ds_test['half_true_counts'] = ds_test['half_true_counts'] + ds_test['barely_true_counts']
ds_validation['half_true_counts'] = ds_validation['half_true_counts'] + ds_validation['barely_true_counts']


In [7]:
# Dropping 'barely_true_counts' from each dataset
ds_train = ds_train.drop('barely_true_counts', axis=1)
ds_test = ds_test.drop('barely_true_counts', axis=1)
ds_validation = ds_validation.drop('barely_true_counts', axis=1)

# Display results
print("Training Dataset:")
display(ds_train.head())
print("\nTest Dataset:")
display(ds_test.head())
print("\nValidation Dataset:")
display(ds_validation.head())

Training Dataset:


,label,statement,false_counts,half_true_counts,mostly_true_counts
0,0,Says the Annies List political group supports ...,1.0,0.0,0.0
1,1,When did the decline of coal start? It started...,0.0,1.0,1.0
2,2,"Hillary Clinton agrees with John McCain ""by vo...",80.0,230.0,163.0
3,0,Health care reform legislation is likely to ma...,63.0,10.0,5.0
4,1,The economic turnaround started at the end of ...,11.0,35.0,19.0



Test Dataset:


,label,statement,false_counts,half_true_counts,mostly_true_counts
0,3,Building a wall on the U.S.-Mexico border will...,48.0,72.0,23.0
1,0,Wisconsin is on pace to double the number of l...,1.0,2.0,0.0
2,0,Says John McCain has done nothing to help the ...,175.0,114.0,37.0
3,1,Suzanne Bonamici supports a plan that will cut...,2.0,4.0,1.0
4,5,When asked by a reporter whether hes at the ce...,14.0,7.0,2.0



Validation Dataset:


,label,statement,false_counts,half_true_counts,mostly_true_counts
0,4,We have less Americans working now than in the...,0.0,2.0,0.0
1,5,"When Obama was sworn into office, he DID NOT u...",148.0,19.0,5.0
2,0,Says Having organizations parading as being so...,1.0,1.0,1.0
3,1,Says nearly half of Oregons children are poor.,1.0,1.0,1.0
4,1,On attacks by Republicans that various program...,80.0,230.0,163.0


# Getting Data and Model ready
## Preparing the data and loading model_helper

In [8]:
# Initialize ModelHelper
model_helper = ModelHelper()

# Preprocess text data
train_texts = ds_train['statement'].apply(model_helper.preprocess_text)
test_texts = ds_test['statement'].apply(model_helper.preprocess_text)
val_texts = ds_validation['statement'].apply(model_helper.preprocess_text)

# Convert texts to sequences
train_sequences = model_helper.preprocess_text(train_texts)
test_sequences = model_helper.preprocess_text(test_texts)
val_sequences = model_helper.preprocess_text(val_texts)

# Get truthfulness columns
truthfulness_columns = model_helper.truthfulness_columns
truthfulness_columns.remove('pants_on_fire_counts')
truthfulness_columns.remove('barely_true_counts')

## Normalizing data

In [9]:
# Get the raw count values for training
train_labels = model_helper.normalize_counts(ds_train)
test_labels = model_helper.normalize_counts(ds_test)
val_labels = model_helper.normalize_counts(ds_validation)

## Create classification model

In [10]:
# Create text classification model
vocab_size = 10000  # Matches max_tokens in preprocess_text
embedding_dim = 100
max_sequence_length = 200
num_classes = len(truthfulness_columns)  # Number of truthfulness categories

model = model_helper.create_text_classification_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim, 
    max_sequence_length=max_sequence_length,
    num_classes=num_classes
)

## Prepare Datasets

In [11]:
# Prepare datasets
train_dataset, val_dataset, test_dataset = model_helper.prepare_datasets(
    train_sequences=train_sequences,
    train_labels=train_labels,
    val_sequences=val_sequences,
    val_labels=val_labels,
    test_sequences=test_sequences,
    test_labels=test_labels,
    batch_size=32
)

## Training the model

In [12]:
%load_ext tensorboard
%tensorboard --logdir models/logs/fit

# Train the model using ModelHelper's train_model method
history = model_helper.train_model(
    model=model,
    train_data=train_dataset,
    validation_data=val_dataset,
    epochs=15,
    batch_size=32
)

Reusing TensorBoard on port 6006 (pid 16988), started 4:33:09 ago. (Use '!kill 16988' to kill it.)

Epoch 1/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 198s 581ms/step - accuracy: 0.3709 - categorical_accuracy: 0.3709 - loss: 1.8313 - precision: 0.7946 - recall: 0.2395 - val_accuracy: 0.5818 - val_categorical_accuracy: 0.5818 - val_loss: 1.3589 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 192s 598ms/step - accuracy: 0.4712 - categorical_accuracy: 0.4712 - loss: 1.3734 - precision: 0.8193 - recall: 0.0960 - val_accuracy: 0.5818 - val_categorical_accuracy: 0.5818 - val_loss: 1.3162 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 191s 595ms/step - accuracy: 0.5514 - categorical_accuracy: 0.5514 - loss: 1.2998 - precision: 0.8154 - recall: 0.0341 - val_accuracy: 0.5818 - val_categorical_accuracy: 0.5818 - val_loss: 1.2689 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 188s 586ms/step - accura

## Evaluate Model

In [13]:
# Evaluate on test set
test_metrics = model.evaluate(test_dataset)
print(test_metrics)

41/41 ━━━━━━━━━━━━━━━━━━━━ 7s 165ms/step - accuracy: 0.5585 - categorical_accuracy: 0.5585 - loss: 1.1607 - precision: 0.0000e+00 - recall: 0.0000e+00
[1.163125991821289, 0.5767731666564941, 0.5767731666564941, 0.0, 0.0]


## Save model

In [14]:
# Save the model
model_helper.save_model(model, "text_classification_model_threecolB")

True

## Load and Use Saved model

In [15]:
# Load the saved model
loaded_model = model_helper.load_model("text_classification_model_threecolB.keras")
test_string = 'The president is a good man'
prediction = loaded_model.predict(model_helper.preprocess_text(test_string))
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
[[0.30115888 0.4581311  0.24071005]]


## Use prepared test data to get Sample Predictions

In [16]:
# Make predictions on test dataset
print(test_dataset)
predictions = loaded_model.predict(test_dataset)
# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Print sample predictions
print("\nSample predictions:")
for i in range(5):
    print(f"Example {i+1}:")
    print(f"Predicted probabilities: {predictions[i]}")
    print(f"Predicted class: {predicted_classes[i]}")
    print(f"Actual class: {np.argmax(test_labels[i])}\n")


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 200), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>
41/41 ━━━━━━━━━━━━━━━━━━━━ 8s 179ms/step

Sample predictions:
Example 1:
Predicted probabilities: [0.30291516 0.4561228  0.24096204]
Predicted class: 1
Actual class: 1

Example 2:
Predicted probabilities: [0.30291516 0.4561228  0.24096204]
Predicted class: 1
Actual class: 1

Example 3:
Predicted probabilities: [0.30291516 0.4561228  0.24096204]
Predicted class: 1
Actual class: 0

Example 4:
Predicted probabilities: [0.30291516 0.4561228  0.24096204]
Predicted class: 1
Actual class: 1

Example 5:
Predicted probabilities: [0.30291516 0.4561228  0.24096204]
Predicted class: 1
Actual class: 0



In [17]:
models_list = model_helper.list_models()
print(models_list)

['logs', '__pycache__']
